In [1]:
# Hack to make the module importable
import sys
sys.path.append(r'./../')

In [2]:
%load_ext autoreload
%autoreload 2
from py2neo import Graph
import requests
import pyodata

from rel2graph.relational_modules.odata import ODataListIterator
from rel2graph import Converter, register_attribute_postprocessor, Attribute
import rel2graph.common_modules

In [3]:
# Configure Logging
import logging

#logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger("rel2graph")
logger.setLevel(logging.DEBUG)
log_formatter = logging.Formatter("%(asctime)s [%(threadName)s]::[%(levelname)s] : %(message)s")
console_handler = logging.StreamHandler()
console_handler.setFormatter(log_formatter)
logger.addHandler(console_handler)

In [4]:
graph = Graph(scheme="http", host="localhost", port=7474,  auth=('neo4j', 'password'))

graph.delete_all()  

In [5]:
###### CONFIGURATIONS #########
SERVICE_URL = 'https://ws.parlament.ch/odata.svc/'
HTTP_LIB = requests.Session()
client = pyodata.Client(SERVICE_URL, HTTP_LIB)

# Initialize node and relation dumps
nodes = []
relations = []

target = client.entity_sets.Person.get_entity(ID=4049,Language="DE").execute()

In [6]:
@register_attribute_postprocessor
def APPEND(attribute, string):
    new_attr = Attribute(attribute.key, attribute.value + string)
    return new_attr

@register_attribute_postprocessor
def ASSERT_EQ(attribute: Attribute, value: str) -> Attribute:
    assert attribute.value == value, f"Attribute {attribute.key} '{attribute.value}' != '{value}'"
    return attribute

2021-12-21 13:50:53,435 [MainThread]::[DEBUG] : Registered attribute postprocessor 'append''.


In [7]:
iterator = ODataListIterator([target])

In [8]:
schema_config = "./odata_demo_config.yaml"
converter = Converter(schema_config, iterator, graph, num_workers=1)

2021-12-21 13:50:54,276 [MainThread]::[DEBUG] : Parsing config file './odata_demo_config.yaml'
2021-12-21 13:50:54,280 [MainThread]::[DEBUG] : Compiling ENTITY("Person") : {'NODE("Person") person': ['+ ID = Person.ID', 'FirstName = Person.FirstName', 'LastName = append(Person.LastName, " I appended something")', 'Astaticattr = "HelloWorld!"'], 'CONDITION(NODE("Language"), "resource[\'FirstName\'] == \'Matthias\'") language': ['ShortName = ASSERT_EQ(Person.NativeLanguage, "D")'], 'RELATION(person, "speaks", language)': ['Since = Person.DateOfBirth']}
2021-12-21 13:50:54,284 [MainThread]::[DEBUG] : Compiling module 'AttributeFactory' with static args ['ID', 'ID']
2021-12-21 13:50:54,285 [MainThread]::[DEBUG] : Requested factory 'AttributeFactory' from registry.
2021-12-21 13:50:54,285 [MainThread]::[DEBUG] : Compiling module 'AttributeFactory' with static args ['FirstName', 'FirstName']
2021-12-21 13:50:54,286 [MainThread]::[DEBUG] : Requested factory 'AttributeFactory' from registry.
20

In [9]:
converter()

2021-12-21 13:50:54,815 [MainThread]::[INFO] : Running convertion with 1 parallel workers.
2021-12-21 13:50:54,816 [MainThread]::[INFO] : Starting creation of nodes.
2021-12-21 13:50:54,818 [Worker-0]::[DEBUG] : Starting Worker 0
2021-12-21 13:50:54,819 [Worker-0]::[DEBUG] : Processing ODataResource 'Person' (ID=4049,Language='DE')
2021-12-21 13:50:54,874 [Worker-0]::[INFO] : DONE
2021-12-21 13:50:54,875 [Worker-0]::[DEBUG] : Exiting Worker 0
2021-12-21 13:50:54,922 [MainThread]::[INFO] : Starting creation of relations.
2021-12-21 13:50:54,923 [Worker-0]::[DEBUG] : Starting Worker 0
2021-12-21 13:50:54,923 [Worker-0]::[DEBUG] : Processing ODataResource 'Person' (ID=4049,Language='DE')
2021-12-21 13:50:54,955 [Worker-0]::[INFO] : DONE
2021-12-21 13:50:54,956 [Worker-0]::[DEBUG] : Exiting Worker 0
2021-12-21 13:50:55,027 [MainThread]::[INFO] : Processed in total 2 nodes and 2 relations (this run took 0s)
